In [1]:
import re
import gc
import pickle
import argparse
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import *
from keras import optimizers
from keras import backend as K
from collections import Counter
from keras.utils import np_utils
from gensim.models import word2vec
from sklearn.metrics import f1_score
from keras.regularizers import l1, l2
from keras.engine import InputSpec, Layer
from keras.models import Model, load_model
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import text, sequence
from keras.layers import Input, Embedding, Dense
from sklearn.preprocessing import StandardScaler
from keras import optimizers, losses, activations
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tqdm import tqdm
from keras.layers.noise import GaussianNoise
# from models_def import Attention
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from keras.callbacks import LearningRateScheduler
config = argparse.Namespace()

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
file_path ="weights_base.best.h5"
config.len_desc = 800000
config.maxlen = 1000
max_features = 800000
vec_len = 600
count_thres = 5

In [3]:
der=False
if der:
    train_dir = '../input/new_data/train_set.csv'
    test_dir = '../input/new_data/test_set.csv'
else:
#     train_dir = '../input/new_data/train_remove60.csv'
#     test_dir = '../input/new_data/test_remove60.csv'
    train_dir = '../input/new_data/shuffle_train.csv'
    test_dir = '../input/new_data/test_set.csv'

train = pd.read_csv(train_dir)
test = pd.read_csv(test_dir)
original_train = pd.read_csv('../input/new_data/train_set.csv')
shuffle_data = train[original_train.shape[0]:]

In [4]:
all_data = pd.concat([train, test])
all_data = all_data.reset_index(drop=True)
train_offset = train.shape[0]

In [5]:
df_y_train = (train["class"]-1).astype(int)
# column = "word_seg"
test_id = test[["id"]].copy()

/usr/local/lib/python3.5/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [6]:
column = "word_seg"

tknzr_word = Tokenizer(num_words=config.len_desc)
tknzr_word.fit_on_texts(all_data[column].values)

In [7]:

low_count_words = [w for w, c in tknzr_word.word_counts.items() if c < count_thres]
# print(len(tknzr_word.texts_to_sequences(all_data[column].values)))
for w in tqdm(low_count_words):
    del tknzr_word.word_index[w]
    del tknzr_word.word_docs[w]
    del tknzr_word.word_counts[w]
# print(len(tknzr_word.texts_to_sequences(all_data[column].values)))

100%|██████████| 897523/897523 [00:01<00:00, 755498.65it/s]


In [8]:
tr_word_seq = tknzr_word.texts_to_sequences(train[column].values)
te_word_seq = tknzr_word.texts_to_sequences(test[column].values)

tr_word_pad = pad_sequences(tr_word_seq, maxlen=config.maxlen)
te_word_pad = pad_sequences(te_word_seq, maxlen=config.maxlen)

In [9]:
from gensim.models import word2vec

EMBEDDING = '../feature/word2vec_file/avito600d.w2v'
model = word2vec.Word2Vec.load(EMBEDDING)
word_index = tknzr_word.word_index
nb_words_desc = min(max_features, len(word_index))
embedding_matrix_desc = np.zeros((nb_words_desc+1, vec_len))
for word, i in word_index.items():
    if i >= max_features: continue
    try:
        embedding_vector = model[word]
    except KeyError:
        embedding_vector = None
    if embedding_vector is not None: embedding_matrix_desc[i] = embedding_vector

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [10]:
Y = df_y_train

In [11]:
one_y_train=np_utils.to_categorical(Y,num_classes=19)

In [12]:
# cvlist2 = list(StratifiedKFold(n_splits=5, random_state=786).split(tr_word_pad, Y))
cvlist2 = list(StratifiedShuffleSplit(n_splits=5, test_size=0.1, random_state=786).split(tr_word_pad, Y))

In [13]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [14]:
class AttentionWeightedAverage(Layer):
    """
    Computes a weighted average of the different channels across timesteps.
    Uses 1 parameter pr. channel to compute the attention value for a single timestep.
    """

    def __init__(self, return_attention=False, **kwargs):
        self.init = initializers.get('uniform')
        self.supports_masking = True
        self.return_attention = return_attention
        super(AttentionWeightedAverage, self).__init__(** kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(ndim=3)]
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[2], 1),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init)
        self.trainable_weights = [self.W]
        super(AttentionWeightedAverage, self).build(input_shape)

    def call(self, x, mask=None):
        # computes a probability distribution over the timesteps
        # uses 'max trick' for numerical stability
        # reshape is done to avoid issue with Tensorflow
        # and 1-dimensional weights
        logits = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))

        # masked timesteps have zero weight
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            ai = ai * mask
        att_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, att_weights]
        return result

    def get_output_shape_for(self, input_shape):
        return self.compute_output_shape(input_shape)

    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0], input_shape[1])]
        return (input_shape[0], output_len)

    def compute_mask(self, input, input_mask=None):
        if isinstance(input_mask, list):
            return [None] * len(input_mask)
        else:
            return None

In [15]:
class AttentionLayer(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
        """
        self.supports_masking = True
        # self.init = initializations.get('glorot_uniform')
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        # eij = K.dot(x, self.W) TF backend doesn't support it

        # features_dim = self.W.shape[0]
        # step_dim = x._keras_shape[1]

        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))),
                        (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        # print weigthted_input.shape
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        # return input_shape[0], input_shape[-1]
        return input_shape[0], self.features_dim

In [16]:
from sklearn.base import BaseEstimator, ClassifierMixin
class GRUClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, gru_dim=150, dense_dim=256, batch_size=128, epochs=2, bidirectional=False, 
                 pool_type='all', initial_weights=None, optimizer='adam' ,verbose=1, out_dim=19, callbacks=None,
                spatial_drop=0.0, dropout=0.0, mask_zero=True, 
                gru_kernel_regularization = 0.0001,
                gru_recurrent_regularization = 0.0001,
                gru_bias_regularization = 0.0001,
                embeddings_regularization = 0.0,
                ):
        
        self.gru_dim = gru_dim
        self.dense_dim = dense_dim
        self.batch_size = batch_size
        self.epochs= epochs
        self.bidirectional = bidirectional
        self.pool_type = pool_type
        self.initial_weights = initial_weights
        self.verbose = verbose
        self.callbacks = callbacks
        self.optimizer = optimizer
        self.out_dim = out_dim
        self.spatial_drop = spatial_drop
        self.dropout = dropout
        self.mask_zero = mask_zero
        self.gru_kernel_regularization = gru_kernel_regularization
        self.gru_recurrent_regularization = gru_recurrent_regularization
        self.gru_bias_regularization = gru_bias_regularization
        self.embeddings_regularization = embeddings_regularization
        
    def _build_model(self):
        inp = Input(shape=(config.maxlen,))
        emb = Embedding(self.initial_weights.shape[0], 
                        600,
                        weights=[self.initial_weights],
                        mask_zero=self.mask_zero,
                        #embeddings_regularizer=regularizers.l2(self.embeddings_regularization),
                        trainable=False)(inp)

        if self.mask_zero:
            emb = ZeroMaskedLayer()(emb)
            
        emb = SpatialDropout1D(self.spatial_drop)(emb)
        
        if self.bidirectional:
            enc = Bidirectional(CuDNNLSTM(int(self.gru_dim), return_sequences=True, return_state=True,
                                         ))(emb)
            x = enc[0]
            state = enc[1]
            enc = Bidirectional(CuDNNGRU(int(self.gru_dim), return_sequences=True, return_state=True,
                                         ))(x)
            x = enc[0]
            state = enc[1]
        else:
            x, state = CuDNNLSTM(int(self.gru_dim), return_sequences=True, return_state=True,
                            kernel_regularizer=regularizers.l2(self.gru_kernel_regularization),
                            recurrent_regularizer=regularizers.l2(self.gru_recurrent_regularization),
                            bias_regularizer=regularizers.l2(self.gru_bias_regularization)
                               )(emb)
            x, state = CuDNNGRU(int(self.gru_dim), return_sequences=True, return_state=True,
                kernel_regularizer=regularizers.l2(self.gru_kernel_regularization),
                recurrent_regularizer=regularizers.l2(self.gru_recurrent_regularization),
                bias_regularizer=regularizers.l2(self.gru_bias_regularization)
                   )(x)

        
        if self.pool_type == 'avg':
            x = GlobalAveragePooling1D()(x)
            x = concatenate([x, state])
            
        elif self.pool_type == 'max':
            x = GlobalMaxPool1D()(x)
            x = concatenate([x, state])
            
        elif self.pool_type == 'attn':
            x = AttentionLayer(config.maxlen)(x)
            x = concatenate([x, state])
            
        elif self.pool_type == 'attn_avg':
            x = AttentionWeightedAverage()(x)
            x = concatenate([x, state])
            
        elif self.pool_type == 'all':
            #x1 = GlobalAveragePooling1D()(emb)
            x2 = GlobalMaxPool1D()(x)
            x3 = AttentionLayer(config.maxlen)(x)
            x4 = Dot(1)([x2, x3])
            x = concatenate([x2, x3, x4])
    
        x = Dropout(self.dropout)(x)
        x = Dense(self.dense_dim)(x)
        x = PReLU()(x)
        
        #x = Dropout(self.dropout)(x)
        #x = Dense(256)(x)
        #x = PReLU()(x)

        out = Dense(self.out_dim, activation="softmax")(x)
        if self.optimizer == 'adam':
            opt = Adam(lr=0.001, decay=0.0,)
        if self.optimizer == 'nadam':
            opt = Nadam(lr=0.002)
        elif self.optimizer == 'rmsprop':
            opt = RMSprop(clipnorm=1.0)
        model = Model(inputs=inp, outputs=out)
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[f1])
        return model
    
    def fit(self, X, y, hold_out_x1, hold_out_y):
        self.model = self._build_model()
        
        if self.callbacks:
            self.model.fit(X, y, batch_size=self.batch_size, epochs=self.epochs,
                       verbose=self.verbose,
                       callbacks=self.callbacks,
                       validation_data=(hold_out_x1, hold_out_y),
                       shuffle=True)
        else:
            self.model.fit(X, y, batch_size=self.batch_size, epochs=self.epochs,
                       verbose=self.verbose,
                        validation_data=(hold_out_x1, hold_out_y),
                       shuffle=True)
        return self
    
    def predict(self, X, y=None):
        if self.model:
            y_hat = self.model.predict(X, batch_size=1024)
        else:
            raise ValueError("Model not fit yet")
        return y_hat

In [17]:
parameter_list = [{'batch_size': [47.0],
  'bidirectional': [1],
  'dense_dim': [973.0],
  'dropout': [0.19862535182199834],
  'gru_bias_reg': [1.011936859273273e-08],
  'gru_dim': [258.0],
  'gru_kernel_reg': [2.0678669679829352e-10],
  'gru_recc_reg': [8.946942716621634e-07],
  'lr1': [0.0015982451490776767],
  'lr2': [0.0002459290205687559],
  'mask_zero': [1],
  'optimizer': [0],
  'pool_type': [3],
  'spatial_drop': [0.3696100622336198]},
 {'batch_size': [83.0],
  'bidirectional': [1],
  'dense_dim': [351.0],
  'dropout': [0.07833431778315075],
  'gru_bias_reg': [1.989216237371643e-09],
  'gru_dim': [278.0],
  'gru_kernel_reg': [2.1606860352426398e-10],
  'gru_recc_reg': [1.6736919208281796e-07],
  'lr1': [0.00263784102869703],
  'lr2': [0.0005711207564167526],
  'mask_zero': [1],
  'optimizer': [0],
  'pool_type': [3],
  'spatial_drop': [0.31401382410917008]},
 {'batch_size': [49.0],
  'bidirectional': [1],
  'dense_dim': [997.0],
  'dropout': [0.19115533803668047],
  'gru_bias_reg': [5.222640591389245e-10],
  'gru_dim': [299.0],
  'gru_kernel_reg': [8.078459790975857e-10],
  'gru_recc_reg': [6.100081276448957e-08],
  'lr1': [0.0019427338445684181],
  'lr2': [0.00010186610979091696],
  'mask_zero': [1],
  'optimizer': [0],
  'pool_type': [3],
  'spatial_drop': [0.32614208466560007]},
 {'batch_size': [41.0],
  'bidirectional': [1],
  'dense_dim': [973.0],
  'dropout': [0.20050865242539928],
  'gru_bias_reg': [1.1451922219328368e-08],
  'gru_dim': [292.0],
  'gru_kernel_reg': [1.0516629869555607e-09],
  'gru_recc_reg': [1.2593577396164419e-06],
  'lr1': [0.0016205788115723873],
  'lr2': [0.00011538601448660545],
  'mask_zero': [1],
  'optimizer': [0],
  'pool_type': [3],
  'spatial_drop': [0.3803897135211322]},
 {'batch_size': [37.0],
  'bidirectional': [1],
  'dense_dim': [237.0],
  'dropout': [0.12273937792021693],
  'gru_bias_reg': [2.7055793227129377e-09],
  'gru_dim': [307.0],
  'gru_kernel_reg': [1.9122269544090935e-09],
  'gru_recc_reg': [1.5269966614646778e-06],
  'lr1': [0.0019545667587842147],
  'lr2': [0.00034205962093229346],
  'mask_zero': [1],
  'optimizer': [0],
  'pool_type': [3],
  'spatial_drop': [0.339366738134983]},
 {'batch_size': [16.0],
  'bidirectional': [1],
  'dense_dim': [783.0],
  'dropout': [0.19251258375962352],
  'gru_bias_reg': [1.6889374260262626e-08],
  'gru_dim': [401.0],
  'gru_kernel_reg': [2.1685591958268602e-10],
  'gru_recc_reg': [8.179324804312695e-07],
  'lr1': [0.0016526011543532724],
  'lr2': [0.00020256532886638333],
  'mask_zero': [1],
  'optimizer': [0],
  'pool_type': [3],
  'spatial_drop': [0.3994866402536531]},
 {'batch_size': [90.0],
  'bidirectional': [1],
  'dense_dim': [995.0],
  'dropout': [0.22199634608987717],
  'gru_bias_reg': [1.162462425352503e-10],
  'gru_dim': [234.0],
  'gru_kernel_reg': [2.6194833614316782e-09],
  'gru_recc_reg': [3.719163247084088e-10],
  'lr1': [0.002927034550684743],
  'lr2': [0.0003517541152030026],
  'mask_zero': [1],
  'optimizer': [0],
  'pool_type': [3],
  'spatial_drop': [0.3670368024684063]},
 {'batch_size': [71.0],
  'bidirectional': [1],
  'dense_dim': [999.0],
  'dropout': [0.31884760544934115],
  'gru_bias_reg': [3.489577142351314e-10],
  'gru_dim': [310.0],
  'gru_kernel_reg': [3.464891994757166e-10],
  'gru_recc_reg': [1.5572206853920122e-09],
  'lr1': [0.0019593704753874588],
  'lr2': [0.000472320376364918],
  'mask_zero': [1],
  'optimizer': [0],
  'pool_type': [3],
  'spatial_drop': [0.3905500602659617]},
 {'batch_size': [21.0],
  'bidirectional': [1],
  'dense_dim': [971.0],
  'dropout': [0.07825915818812121],
  'gru_bias_reg': [9.370609019263161e-08],
  'gru_dim': [452.0],
  'gru_kernel_reg': [1.1699932655152522e-09],
  'gru_recc_reg': [1.118300667865804e-06],
  'lr1': [0.0009194436554174992],
  'lr2': [0.00011484392164348851],
  'mask_zero': [1],
  'optimizer': [0],
  'pool_type': [4],
  'spatial_drop': [0.5127687159502792]},
 {'batch_size': [41.0],
  'bidirectional': [1],
  'dense_dim': [888.0],
  'dropout': [0.18388910768345024],
  'gru_bias_reg': [1.0198714543301587e-08],
  'gru_dim': [322.0],
  'gru_kernel_reg': [4.3029409259060245e-08],
  'gru_recc_reg': [3.1764731171976037e-06],
  'lr1': [0.0017624941021206772],
  'lr2': [0.0006933312845721429],
  'mask_zero': [1],
  'optimizer': [0],
  'pool_type': [2],
  'spatial_drop': [0.4120345974541232]}
                 ]

In [18]:
decay_dic = {0:0.001, 1:0.001, 2:0.0009, 3:0.0008, 4:0.0007, 5:0.0006, 6:0.0005, 7:0.0004, 8:0.0003, 9:0.0002, 10:0.0001,
            11:0.00009, 12:0.00008, 13:0.00007, 14:0.00006, 15:0.00005}
def lr_decay(epoch):
    return decay_dic[epoch]
def shuffle_train_predict(model, cvlist, X, y, X_test, lr_decay):
    y_trues = []
    y_preds = []
    y_test_preds = []
    scores = []
    now_nfold=0
    LRDecay = LearningRateScheduler(lr_decay)
    shuffle_word_pad = tr_word_pad[original_train.shape[0]:]
    shuffle_word_pad_y = one_y_train[original_train.shape[0]:]
    split_y = Y[original_train.shape[0]:]
#     kf = KFold(n_splits=5, shuffle=False, random_state=233)
    cvs = list(StratifiedShuffleSplit(n_splits=5, test_size=0.1, random_state=786).split(shuffle_data, split_y))
    for tr_index, val_index in cvlist:
        now_nfold+=1
        print ("now is {} fold".format(now_nfold))
        X_tr, y_tr = X[tr_index, :], y[tr_index, :]
        X_val, y_val = X[val_index, :], y[val_index, :]
        shuffle_number = 1
        for shuffle_train_index, shuffle_test_index in cvs:
            shuffle_train = shuffle_word_pad[shuffle_train_index]
            shuffle_train_y = shuffle_word_pad_y[shuffle_train_index]
            if shuffle_number == now_nfold:
                break
            shuffle_number+=1
        X_tr = np.vstack((X_tr, shuffle_train))
        y_tr = np.vstack((y_tr, shuffle_train_y))
        checkpoint = ModelCheckpoint(file_path, save_best_only=True,verbose=1, monitor='val_f1',  mode='max')
        early = EarlyStopping( monitor='val_f1',  mode='max', patience=2,)

        model.set_params(**{'callbacks':[checkpoint,early,LRDecay]})
        model.fit(X_tr, y_tr, X_val, y_val)

        y_pred = model.predict(X_val)
        y_test_pred = model.predict(X_test)
#         score = f1_score(y_val, y_pred,  average='macro')
#         scores.append(score)
#         print("f1 for this fold is ", score)
        y_trues.append(y_val)
        y_preds.append(y_pred)
        y_test_preds.append(y_test_pred)
        K.clear_session()
        gc.collect()
        #break
    y_trues = np.concatenate(y_trues)
    y_preds = np.concatenate(y_preds)
    y_test_preds = np.mean(y_test_preds, axis=0)
    print("Shape of test _preds is ", y_test_preds.shape)
    print("Means of val and test preds are {} and {}".format(np.mean(y_preds, axis=1), np.mean(y_test_preds, axis=1)))
    score = f1_score(y_trues, y_preds,  average='macro')
    print("Overall score on 10 fold CV is {}".format(score))
    
    return y_preds, y_trues, y_test_preds

In [19]:
def train_predict(parameter_space):
    def lr_decay(epoch):
        if epoch == 0:
            return parameter_space['lr1'][0]
        else:
            return parameter_space['lr2'][0]
    model = GRUClassifier(initial_weights=embedding_matrix_desc, bidirectional=[False, True][parameter_space['bidirectional'][0]],
                          gru_dim = int(parameter_space['gru_dim'][0]),
                          dense_dim = int(parameter_space['dense_dim'][0]),
                          mask_zero = [True, False][parameter_space['mask_zero'][0]],
                          pool_type = ['avg', 'max', 'attn',  'all', 'attn_avg'][parameter_space['pool_type'][0]],
                          batch_size= int(parameter_space['batch_size'][0]), 
                          epochs=15,
                          optimizer=["adam", "rmsprop"][parameter_space['optimizer'][0]],
                          dropout=parameter_space['dropout'][0],
                          spatial_drop=parameter_space['spatial_drop'][0],
                          gru_kernel_regularization = parameter_space["gru_kernel_reg"][0],
                          gru_recurrent_regularization = parameter_space["gru_recc_reg"][0],
                          gru_bias_regularization = parameter_space["gru_bias_reg"][0],
                          #embeddings_regularization = parameter_space["embeddings_reg"],
                          )

    y_preds, y_trues, y_test_preds = shuffle_train_predict(model, cvlist2, tr_word_pad, one_y_train, te_word_pad, lr_decay) 
    return y_preds, y_trues, y_test_preds

In [20]:
y_preds_all = []
y_trues_all = []
y_test_preds_all = []
rounds = 1
for params in parameter_list:
    y_preds, y_trues, y_test_preds = train_predict(params)
    y_preds_all.append(y_preds)
    y_trues_all.append(y_trues)
    y_test_preds_all.append(y_test_preds)
    with open('../feature/rnn_model_bagging{}'.format(rounds),'wb') as fout:
        pickle.dump([y_preds,y_trues,y_test_preds],fout)
    rounds+=1

now is 1 fold
Train on 111667 samples, validate on 11318 samples
Epoch 1/15
  3431/111667 [..............................] - ETA: 52:33 - loss: 2.0284 - f1: 0.2879

KeyboardInterrupt: 

In [ ]:
with open('../feature/rnn_model_bagging_final','wb') as fout:
    pickle.dump([y_preds_all,y_trues_all,y_test_preds_all],fout)